# Assignment Chapter 2 - DEEP LEARNING [Case #4]
Startup Campus, Indonesia - `Artificial Intelligence (AI)` (Batch 7)
* Task: **CLASSIFICATION**
* DL Framework: **PyTorch**
* Dataset: Credit Card Fraud 2023
* Libraries: Pandas/cuDF, Scikit-learn/cuML, Numpy/cuPy
* Objective: Classify credit fraud transactions using Multilayer Perceptron

`PERSYARATAN` Semua modul (termasuk versi yang sesuai) sudah di-install dengan benar.
<br>`CARA PENGERJAAN` Lengkapi baris kode yang ditandai dengan **#TODO**.
<br>`TARGET PORTFOLIO` Peserta mampu mengklasifikasi transaksi fraud menggunakan *Multilayer Perceptron*

### Import Libraries

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 530, done.
remote: Counting objects: 100% (261/261), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 530 (delta 171), reused 129 (delta 94), pack-reused 269 (from 1)
Receiving objects: 100% (530/530), 170.56 KiB | 998.00 KiB/s, done.
Resolving deltas: 100% (273/273), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.3 MB/s eta 0:00:00
Installing RAPIDS remaining 24.6.* libraries
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 478.0/478.0 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 GB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.4/141.4 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.7/895.7 kB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import shutil
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

<font color="red">**- - - - MOHON DIPERHATIKAN - - - -**</font>
<br>**Aktifkan GPU sekarang.** Di Google Colab, klik **Runtime > Change Runtime Type**, lalu pilih **T4 GPU**.

### Dataset Loading (CPU vs. GPU)

In [ ]:
from pandas import read_csv as read_by_CPU
from cudf import read_csv as read_by_GPU

In [ ]:
# unzip the file
shutil.unpack_archive('dataset_case_04.zip', '/content/sample_data/', 'zip')

In [ ]:
# TODO: Impor dataset dengan Pandas, gunakan fungsi "read_by_CPU"
%time data_cpu = read_by_CPU('/content/sample_data/creditcard_2023.csv')

CPU times: user 6.32 s, sys: 329 ms, total: 6.65 s
Wall time: 10.3 s


In [ ]:
# Impor dataset dengan cuDF (Pandas di GPU)
%time data_gpu = read_by_GPU('/content/sample_data/creditcard_2023.csv')

# Menampilkan data untuk verifikasi
data_gpu.head()

CPU times: user 297 ms, sys: 497 ms, total: 793 ms
Wall time: 1.5 s


,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-0.260648,-0.469648,2.496266,-0.083724,0.129681,0.732898,0.519014,-0.130006,0.727159,...,-0.110552,0.217606,-0.134794,0.165959,0.126280,-0.434824,-0.081230,-0.151045,17982.10,0
1,1,0.985100,-0.356045,0.558056,-0.429654,0.277140,0.428605,0.406466,-0.133118,0.347452,...,-0.194936,-0.605761,0.079469,-0.577395,0.190090,0.296503,-0.248052,-0.064512,6531.37,0
2,2,-0.260272,-0.949385,1.728538,-0.457986,0.074062,1.419481,0.743511,-0.095576,-0.261297,...,-0.005020,0.702906,0.945045,-1.154666,-0.605564,-0.312895,-0.300258,-0.244718,2513.54,0
3,3,-0.152152,-0.508959,1.746840,-1.090178,0.249486,1.143312,0.518269,-0.065130,-0.205698,...,-0.146927,-0.038212,-0.214048,-1.893131,1.003963,-0.515950,-0.165316,0.048424,5384.44,0
4,4,-0.206820,-0.165280,1.527053,-0.448293,0.106125,0.530549,0.658849,-0.212660,1.049921,...,-0.106984,0.729727,-0.161666,0.312561,-0.414116,1.071126,0.023712,0.419117,14278.97,0


In [ ]:
# TODO: Hilangkan kolom ID
data_gpu = data_gpu.drop(columns=['id'])

# Menampilkan data untuk verifikasi
data_gpu.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-0.260648,-0.469648,2.496266,-0.083724,0.129681,0.732898,0.519014,-0.130006,0.727159,0.637735,...,-0.110552,0.217606,-0.134794,0.165959,0.126280,-0.434824,-0.081230,-0.151045,17982.10,0
1,0.985100,-0.356045,0.558056,-0.429654,0.277140,0.428605,0.406466,-0.133118,0.347452,0.529808,...,-0.194936,-0.605761,0.079469,-0.577395,0.190090,0.296503,-0.248052,-0.064512,6531.37,0
2,-0.260272,-0.949385,1.728538,-0.457986,0.074062,1.419481,0.743511,-0.095576,-0.261297,0.690708,...,-0.005020,0.702906,0.945045,-1.154666,-0.605564,-0.312895,-0.300258,-0.244718,2513.54,0
3,-0.152152,-0.508959,1.746840,-1.090178,0.249486,1.143312,0.518269,-0.065130,-0.205698,0.575231,...,-0.146927,-0.038212,-0.214048,-1.893131,1.003963,-0.515950,-0.165316,0.048424,5384.44,0
4,-0.206820,-0.165280,1.527053,-0.448293,0.106125,0.530549,0.658849,-0.212660,1.049921,0.968046,...,-0.106984,0.729727,-0.161666,0.312561,-0.414116,1.071126,0.023712,0.419117,14278.97,0


### Standardization (CPU vs. GPU)

In [ ]:
from sklearn.preprocessing import StandardScaler as StandardScalerCPU
from cuml.preprocessing import StandardScaler as StandardScalerGPU

In [ ]:
ScalerCPU = StandardScalerCPU()
ScalerGPU = StandardScalerGPU()

arbitrary_features = ["V"+str(i+1) for i in range(27)]

In [ ]:
%%time

data_cpu[arbitrary_features] = ScalerCPU.fit_transform(data_cpu[arbitrary_features].values)
data_cpu["Amount"] = ScalerCPU.fit_transform(data_cpu["Amount"].values.reshape(-1, 1)).squeeze()

CPU times: user 222 ms, sys: 482 ms, total: 704 ms
Wall time: 691 ms


In [ ]:
%%time

data_gpu[arbitrary_features] = ScalerGPU.fit_transform(data_gpu[arbitrary_features].values)
data_gpu["Amount"] = ScalerGPU.fit_transform(data_gpu["Amount"].values.reshape(-1, 1)).squeeze()

CPU times: user 18.8 s, sys: 1.15 s, total: 19.9 s
Wall time: 24.4 s


### Train/Test Split (CPU vs. GPU)

In [ ]:
from sklearn.model_selection import train_test_split as splitCPU
from cuml.preprocessing import train_test_split as splitGPU

In [ ]:
# TODO: Tentukan X (features) dan Y (target), gunakan "data_gpu"
# Asumsikan kolom 'Class' sebagai target (Y) dan sisanya adalah fitur (X)
X = data_gpu.drop(columns=['Class'])
Y = data_gpu['Class']

In [ ]:
%%time

# TODO: Pecah dataset dengan komposisi 80% train set dan 20% test set, dengan fungsi "splitCPU"
test_size = 0.2
random_state = 42
x_train, x_test, y_train, y_test = splitCPU(X, Y, test_size=test_size, random_state=random_state)

print("x_train shape: ", x_train.shape)
print("x_test shape: ", x_test.shape)

x_train shape:  (454904, 29)
x_test shape:  (113726, 29)
CPU times: user 53.2 ms, sys: 16.6 ms, total: 69.8 ms
Wall time: 141 ms


In [ ]:
%%time

# TODO: Lakukan hal yang sama untuk data spliting, tetapi dengan fungsi "splitGPU"
test_size =  0.2
random_state = 42
x_train, x_test, y_train, y_test = splitGPU(X, Y, test_size=test_size, random_state=random_state)

print("x_train shape: ", x_train.shape)
print("x_test shape: ", x_test.shape)

x_train shape:  (454904, 29)
x_test shape:  (113726, 29)
CPU times: user 1.15 s, sys: 71.4 ms, total: 1.22 s
Wall time: 1.28 s


### Convert the dataset into Tensor

In [ ]:
import cupy # Numpy for GPU

In [ ]:
# TODO: Aktifkan GPU (CUDA) sebagai device untuk training
# device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
device = torch.device('cuda')
# buat ngecek aja
# device
print(f"Device yang Digunakan: {device}")

Device yang Digunakan: cuda


In [ ]:
x_train_tensor = torch.tensor(x_train.to_cupy(), dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.to_cupy(), dtype=torch.float32).to(device)

x_test_tensor = torch.tensor(x_test.to_cupy(), dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.to_cupy(), dtype=torch.float32).to(device)

Train_tensor = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
Test_tensor = torch.utils.data.TensorDataset(x_test_tensor, y_test_tensor)

### Batching the Dataset with PyTorch DataLoader

In [ ]:
# TODO: Tentukan nilai batch
# batch_size = 32
batch_size = 64
# batch_size = 128

Train_dataset = DataLoader(Train_tensor, batch_size=batch_size, shuffle=True)
Test_dataset = DataLoader(Test_tensor, batch_size=batch_size, shuffle=False)

### Model Blueprint

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, input_dim, num_neurons):
        super(FeedForward, self).__init__()
        self.input_dim = input_dim
        self.num_neurons = num_neurons

        self.net = nn.Sequential(
            nn.Linear(self.input_dim, self.num_neurons),
            nn.ReLU()
        )

    def forward(self, x):
        return self.net(x)

    def to(self, device):
        self.net.to(device)
        return self

class Net(nn.Module):
    def __init__(self, in_features, num_layers, num_neurons):
        super(Net, self).__init__()
        self.in_features = in_features
        self.num_layers = num_layers
        self.num_neurons = num_neurons

        self.fc1 = nn.Linear(self.in_features, self.num_neurons)
        self.relu = nn.ReLU()
        self.blocks = [FeedForward(self.num_neurons, self.num_neurons).to(device) \
                       for _ in range(self.num_layers)]
        self.output_layer = nn.Linear(self.num_neurons, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        output = self.relu(self.fc1(x))

        for block in self.blocks:
            output = block(output)
        output = self.sigmoid(self.output_layer(output))

        return output

### Model Hyperparameters and Parameters

In [ ]:
# [ PERTANYAAN ]
# Apa perbedaan hyperparameters dan parameters?

### Hyperparameters
**Hyperparameters** merupakan pengaturan model yang ditentukan sebelum proses pelatihan dimulai. Mereka tidak dipelajari oleh model itu sendiri, melainkan dipilih oleh suatu *data scientist* atau *data engineer*. Hyperparameters berperan sebagai semacam "**kontrol**" atas proses pembelajaran dalam model.

**Contoh Hyperparameters:**<br>
- Learning Rate<br>
=> Ukuran langkah yang diambil saat mengupdate bobot model.
- Jumlah Hidden Layer<br>
=> Banyaknya lapisan tersembunyi dalam jaringan neural.
- Jumlah Neuron Per-layer<br>
=> Banyaknya neuron dalam setiap lapisan tersembunyi.
- Fungsi Aktivasi<br>
=> Fungsi yang digunakan untuk memperkenalkan non-linearitas ke dalam model.
- Optimizer<br>
=> Algoritma yang digunakan untuk meminimalkan *fungsi loss*.

**Peran Hyperparameters:**<br>
- Memengaruhi Kinerja Model<br>
=> Pilihan hyperparameters yang tepat dapat secara signifikan memengaruhi akurasi, kecepatan konvergensi, dan kemampuan generalisasi model.
- Mengontrol Kompleksitas Model<br>
=> Hyperparameters dapat digunakan untuk mengatur kompleksitas model, sehingga dapat menghindari *overfitting* ataupun *underfitting*.
- Membentuk Arsitektur Model<br>
=> Hyperparameters menentukan struktur dasar dari model neural.

---

### Parameters
**Parameters** merupakan variabel yang dipelajari oleh model selama proses pelatihan. Mereka adalah bobot dan bias yang menghubungkan neuron dalam jaringan neural. Dimana, **bobot (weight)** akan menentukan kekuatan koneksi antara neuron, sedangkan **bias** akan menentukan nilai *offset* dari neuron.

**Contoh Parameters:**<br>
- Bobot (Weights)<br>
=> Nilai numerik yang menentukan seberapa kuat sinyal dari satu neuron diteruskan ke neuron lainnya.
- Bias (Biases)<br>
=> Nilai konstan yang ditambahkan ke output neuron sebelum diteruskan ke lapisan berikutnya.

**Peran Parameters:**<br>
- Merepresentasikan Pengetahuan Model<br>
=> Parameter model berisi informasi yang dipelajari dari data pelatihan.
- Digunakan untuk Membuat Prediksi<br>
=> Parameter digunakan untuk membuat prediksi pada data yang baru.

---

### Kesimpulan Perbedaan Hyperparameters x Parameters

| Fitur | Hyperparameters | Parameters |
|----------|----------|----------|
| Ditetapkan | Sebelum Pelatihan | Selama Pelatihan |
| Diubah | Secara Manual oleh Manusia | Secara Otomatis oleh Algoritma Optimasi |
| Peran | Mengontrol Proses Pembelajaran, Memengaruhi Kinerja Model | Merepresentasikan Pengetahuan yang Dipelajari dari Data dan Digunakan untuk Prediksi |
| Dipelajari dari Data | YA | TIDAK |
| Contoh | Learning Rate, Jumlah Layer, Fungsi Aktivasi | Bobot dan Bias |

In [ ]:
# TODO: Tentukan hyperparameters
epochs = 50
num_layers = 3
num_neurons = 128
learning_rate = 0.003

# Buat Verifikasi Hyperparameters Saja
print(f"Epochs: {epochs}")
print(f"Number of layers: {num_layers}")
print(f"Number of neurons per layer: {num_neurons}")
print(f"Learning rate: {learning_rate}")

Epochs: 50
Number of layers: 3
Number of neurons per layer: 128
Learning rate: 0.003


In [ ]:
# TODO: Tentukan besaran input untuk model
num_inputs = 29 # Jumlah features dalam dataset (selain target)

model = Net(in_features= num_inputs, num_layers=num_layers, num_neurons=num_neurons)
model = model.to(device)

# Verifikasi model dan input size
print(f"Number of inputs (features): {num_inputs}")
print(model)

Number of inputs (features): 29
Net(
  (fc1): Linear(in_features=29, out_features=128, bias=True)
  (relu): ReLU()
  (output_layer): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [ ]:
# Set the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCELoss()

In [ ]:
# Check the number of parameters
print("Number of parameters: {:,}".format(sum(p.numel() for p in model.parameters() if p.requires_grad)))
print("Number of trainable parameters: {:,}".format(sum(p.numel() for p in model.parameters() if p.requires_grad)))

Number of parameters: 3,969
Number of trainable parameters: 3,969


In [ ]:
# [ PERTANYAAN ]
# Mengapa total "trainable parameters" sama dengan total keseluruhan parameter?

Total trainable parameters sama dengan keseluruhan parameter karena seluruh bobot dan bias dalam model dilibatkan dalam proses optimasi selama pelatihan. Oleh karena itu, total **parameters** dalam model (3.969) sama dengan jumlah **trainable parameters** (3,969), karena semuanya dioptimalkan selama proses pembelajaran. Berikut ini beberapa alasan mengapa keduanya dianggap setara:

1.) **Definisi Parameter**<br>
==> Dalam neural network, parameter merujuk pada **berat (weights)** dan **bias** yang terhubung di antara neuron-neuron. Parameter-parameter ini menentukan bagaimana input ditransformasikan saat melewati lapisan-lapisan jaringan.<br>
2.) **Trainable Parameters**<br>
==> Parameter yang dioptimalkan selama training menggunakan metode ***backpropagation*** dan algoritma optimasi seperti **Stochastic Gradient Descent (SGD)** ataupun **ADAM**, dan lain sebagainya. Semua parameter yang mengalami pembaruan dalam model merupakan "**trainable**". <br>
3.) **Relationship (Hubungan)**<br>
==> Kebanyakan model deep learning, pada seluruh parameter yang didefinisikan di lapisan-lapisan jaringan akan mengalami **pembaruan** selama training, sehingga jumlah ***trainable parameters*** akan sama dengan jumlah ***keseluruhan parameter***. <br>
4.) **Pengecualian terdapat Parameter yang Tidak Dilatih**<br>
==> Dalam kasus khusus, mungkin terdapat parameter yang tidak dilatih (misalnya parameter-parameter yang dibekukan dalam *transfer learning*), maka jumlah ***trainable parameters*** dapat lebih kecil daripada total keseluruhan parameter. Namun, secara *default*, untuk model yang baru dilatih dari awal (scratch), semua parameter merupakan **trainable**, sehingga jumlah keduanya akan sama. <br>

Dengan demikian, dalam situasi secara umum (general), seluruh parameter dapat dianggap sebagai **trainable**, yang menjelaskan mengapa ***trainable parameters*** akan sama dengan total ***keseluruhan parameter***.

### Train the Model

In [ ]:
print("Start training ...")
for epoch in range(epochs):
    train_loss = 0.0
    model.train()

    for data, label in Train_dataset:
        data = data.to(device)
        label = label.squeeze()
        label = label.to(device)
        optimizer.zero_grad()
        output = model.forward(data.float())

        loss = criterion(output.squeeze(), label.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss = train_loss / len(Train_dataset.dataset)
    if(epoch % 10 == 0):
        print('  - Epoch: {} \tTraining_loss: {:.6f}'.format(epoch, train_loss))

Start training ...
  - Epoch: 0 	Training_loss: 0.000593
  - Epoch: 10 	Training_loss: 0.000017
  - Epoch: 20 	Training_loss: 0.000011
  - Epoch: 30 	Training_loss: 0.000008
  - Epoch: 40 	Training_loss: 0.000007


### Model ACCURACY should reach >= 95%

In [ ]:
# TODO: Jika akurasi masih dibawah 95%, silakan lakukan fine-tuning

In [ ]:
correct_preds = 0
total_samples = 0

with torch.no_grad():
    for data, labels in Test_dataset:
        labels = labels.squeeze()
        output = model.forward(data.float())
        output = output.squeeze(1)

        predictions = (output >= 0.5).float()
        correct_preds += (predictions == labels).sum().item()
        total_samples += labels.numel()

accuracy = correct_preds / total_samples
print("Model accuracy: {:.2f}%".format(accuracy*100))

Model accuracy: 99.97%


### Scoring
Total `#TODO` = 12
<br>Checklist:

- [ ] Impor dataset dengan Pandas, gunakan fungsi "read_by_CPU"
- [ ] Hilangkan kolom ID
- [ ] Tentukan X (features) dan Y (target), gunakan "data_gpu"
- [ ] Pecah dataset dengan komposisi 80% train set dan 20% test set, dengan fungsi "splitCPU"
- [ ] Lakukan hal yang sama untuk data spliting, tetapi dengan fungsi "splitGPU"
- [ ] Aktifkan GPU (CUDA) sebagai device untuk training
- [ ] Tentukan nilai batch
- [ ] PERTANYAAN: Apa perbedaan hyperparameters dan parameters?
- [ ] Tentukan hyperparameters
- [ ] Tentukan besaran input untuk model
- [ ] PERTANYAAN: Mengapa total "trainable parameters" sama dengan total keseluruhan parameter?
- [ ] Jika akurasi masih dibawah 95%, silakan lakukan fine-tuning

### Additional readings
- N/A

### Copyright © 2024 Startup Campus, Indonesia
* Prepared by **Nicholas Dominic, M.Kom.** [(profile)](https://linkedin.com/in/nicholas-dominic)
* You may **NOT** use this file except there is written permission from PT. Kampus Merdeka Belajar (Startup Campus).
* Please address your questions to mentors.